# Prototype for integration of descriptive and analysis model using Jupyter

## Build Environment
pip install sparqlwrapper

## Run Fuseki Endpoint using OML-Vision GUI

1. Run build
2. Run load

## Setup SPARQL

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd


sparql = SPARQLWrapper(
    "http://localhost:3030/tutorial2/sparql"
)
sparql.setReturnFormat(JSON)



## Get Configuration & Mass Properties

### SPARQL Code

In [2]:
configuration_root_iri = """http://example.com/tutorial2/description/components#orbiter-spacecraft"""

query_string = """
PREFIX mission:     <http://imce.jpl.nasa.gov/foundation/mission#>
PREFIX base:        <http://imce.jpl.nasa.gov/foundation/base#>
PREFIX project:     <http://imce.jpl.nasa.gov/foundation/project#>
PREFIX vim4:		<http://bipm.org/jcgm/vim4#>

SELECT DISTINCT ?c1 ?c1_instancename ?c1_id ?c1_name ?c1_mass ?c2 ?c2_instancename ?c2_id ?c2_name ?c2_mass ?c3 ?c3_instancename ?c3_id ?c3_name ?c4
WHERE {

  VALUES ?c1 { <$configuration_root_iri> }


   OPTIONAL{ 
    ?c1 base:hasIdentifier ?c1_id ;
      base:hasCanonicalName ?c1_name ;
  }
 	OPTIONAL {
		?c1_mass_mag vim4:characterizes ?c1 ;
			vim4:hasDoubleNumber ?c1_mass .
	} 
  OPTIONAL{
    ?c1 base:contains ?c2 ;
    OPTIONAL{
      ?c2 base:hasIdentifier ?c2_id ;
          base:hasCanonicalName ?c2_name .
    }
 	OPTIONAL {
		?c2_mass_mag vim4:characterizes ?c2 ;
			vim4:hasDoubleNumber ?c2_mass .
	}     
    OPTIONAL{
      ?c2 project:isSuppliedBy ?c3 ;
          OPTIONAL{
            ?c3 base:hasIdentifier ?c3_id ;
                base:hasCanonicalName ?c3_name;
                project:isAuthorizedBy ?c4 .
          }          
    }    
  }


  BIND(STRAFTER(STR(?c1), "#") AS ?c1_instancename) .
  BIND(STRAFTER(STR(?c2), "#") AS ?c2_instancename) .
  BIND(STRAFTER(STR(?c3), "#") AS ?c3_instancename) .
 }
ORDER BY ?c2_id

"""


query_string = query_string.replace("$configuration_root_iri", configuration_root_iri)

sparql.setQuery(query_string)



try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)

<urlopen error [Errno 111] Connection refused>


### Tidy Data

In [5]:
# Convert JSON Dictionary to Pandas dataframe
df_query = pd.json_normalize(ret["results"]["bindings"])
#print(df["c2_name.value"], df["c2_mass.value"])
#print(df["c2_name.value"][1], df["c2_mass.value"][1])

## Binding Variables: to Python Parameter Table

### SPARQL CODE

In [6]:

query_string = """
PREFIX base:        <http://imce.jpl.nasa.gov/foundation/base#>
PREFIX mission:        <http://imce.jpl.nasa.gov/foundation/mission#>
PREFIX analysis:        <http://imce.jpl.nasa.gov/foundation/analysis#>
PREFIX sa:        <http://example.com/tutorial2/vocabulary/stateanalysis#>

SELECT DISTINCT ?iri ?statevariable ?value
WHERE {

#  VALUES ?analysisTarget {<http://example.com/tutorial2/description/statedictionary#orbiter-spacecraft.delta-v>}
   VALUES ?analysisTarget {<http://example.com/tutorial2/description/statedictionary#orbiter-spacecraft.mass.wet>}
  VALUES ?componentType { sa:StateVariable }
  VALUES ?relationType { sa:isAffectedBy }

  #	Recursive Query to get state variables
  ?analysisTarget sa:isAffectedBy* ?iri ;
#  ?iri a ?componentType
  
  OPTIONAL{
    ?iri sa:hasStateValue ?value;
         }



  BIND(STRAFTER(STR(?iri), "#") AS ?statevariable) .
  BIND(STRAFTER(STR(?targetIri), "#") AS ?target) .
  BIND(STRAFTER(STR(?cpIri), "#") AS ?component) .
 }
ORDER BY ?iri
"""


sparql.setQuery(query_string)



try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print(r)
except Exception as e:
    print(e)



df_query_param = pd.json_normalize(ret["results"]["bindings"])

{'iri': {'type': 'uri', 'value': 'http://example.com/tutorial2/description/statedictionary#orbiter-spacecraft.delta-v'}, 'statevariable': {'type': 'literal', 'value': 'orbiter-spacecraft.delta-v'}}
{'iri': {'type': 'uri', 'value': 'http://example.com/tutorial2/description/statedictionary#orbiter-spacecraft.initial-orbit'}, 'statevariable': {'type': 'literal', 'value': 'orbiter-spacecraft.initial-orbit'}, 'value': {'type': 'literal', 'value': '400'}}
{'iri': {'type': 'uri', 'value': 'http://example.com/tutorial2/description/statedictionary#orbiter-spacecraft.isp'}, 'statevariable': {'type': 'literal', 'value': 'orbiter-spacecraft.isp'}, 'value': {'type': 'literal', 'value': '350'}}
{'iri': {'type': 'uri', 'value': 'http://example.com/tutorial2/description/statedictionary#orbiter-spacecraft.mass.dry'}, 'statevariable': {'type': 'literal', 'value': 'orbiter-spacecraft.mass.dry'}}
{'iri': {'type': 'uri', 'value': 'http://example.com/tutorial2/description/statedictionary#orbiter-spacecraft.

### Binding to Pandas Dataframe


In [7]:
mapping = [["dv", "orbiter-spacecraft.delta-v"],
  ["initOrbit", "orbiter-spacecraft.initial-orbit"],
  ["I_sp","orbiter-spacecraft.isp"],
  ["m_dry","orbiter-spacecraft.mass.dry"],     
  ["m_fuel","orbiter-spacecraft.mass.fuel"],
  ["m_wet","orbiter-spacecraft.mass.wet"],
  ["targetOrbit","orbiter-spacecraft.target-orbit"]
]

column_names = ["parameter", "statevariable.value"]
mapping_df = pd.DataFrame(mapping, columns=column_names)
merged_df = pd.merge(df_query_param, mapping_df, left_on='statevariable.value', right_on='statevariable.value', how='left')

df_parameters = merged_df[['iri.value','statevariable.value','value.value','parameter']]
df_parameters.columns = ["iri", "statevariable", "value", "parameter"]

df_parameters["index"]= 0

/tmp/ipykernel_11643/3073877374.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_parameters["index"]= 0


In [8]:
df_parameters


,iri,statevariable,value,parameter,index
0,http://example.com/tutorial2/description/state...,orbiter-spacecraft.delta-v,NaN,dv,0
1,http://example.com/tutorial2/description/state...,orbiter-spacecraft.initial-orbit,400,initOrbit,0
2,http://example.com/tutorial2/description/state...,orbiter-spacecraft.isp,350,I_sp,0
3,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.dry,NaN,m_dry,0
4,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.fuel,NaN,m_fuel,0
5,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.wet,NaN,m_wet,0
6,http://example.com/tutorial2/description/state...,orbiter-spacecraft.target-orbit,35786,targetOrbit,0


In [9]:
df_parameters_before = df_parameters
df_parameters = df_parameters.pivot(index="index", columns="parameter", values=["value"])
df_parameters = df_parameters.reset_index(drop=True)
df_parameters = df_parameters["value"]

In [10]:
df_parameters
# df_parameters["I_sp"]

parameter,I_sp,dv,initOrbit,m_dry,m_fuel,m_wet,targetOrbit
0,350,NaN,400,NaN,NaN,NaN,35786


## MassRollUp


In [11]:
import rpy2


### Load “R magic” extension
From now on, code cells starting with %%R will see their content evaluated as R code

In [12]:
%load_ext rpy2.ipython


### Setup R 

In [13]:
%%R

library(reactable)
library(stringr)

searchDirectory <- function(iteration, pattern, parent_directory){
  for(i in 1:iteration){
      path <- list.files(parent_directory, recursive = TRUE, pattern = pattern, full.names = TRUE)
      if(length(path)){
        return(path)
      }
      parent_directory <- dirname(parent_directory)
  }
  print("file not found")
  return(path)
}

source(searchDirectory(4, "osr_common.R", (getwd())))
source(searchDirectory(4, "massRollupKepler16b.R", (getwd())))


Error in library(reactable) : there is no package called ‘reactable’


RInterpreterError: Failed to parse and evaluate line '\nlibrary(reactable)\nlibrary(stringr)\n\nsearchDirectory <- function(iteration, pattern, parent_directory){\n  for(i in 1:iteration){\n      path <- list.files(parent_directory, recursive = TRUE, pattern = pattern, full.names = TRUE)\n      if(length(path)){\n        return(path)\n      }\n      parent_directory <- dirname(parent_directory)\n  }\n  print("file not found")\n  return(path)\n}\n\nsource(searchDirectory(4, "osr_common.R", (getwd())))\nsource(searchDirectory(4, "massRollupKepler16b.R", (getwd())))\n'.
R error message: 'Error in library(reactable) : there is no package called ‘reactable’'

### Binding Variables : toR

query results of the system configurations

In [12]:
%%R -i df_query

# now we can use df_query from python as a dataframe in R

# rename
df_config <- data.frame(
    c1 = df_query$c1.value,
    c1_instancename = df_query$c1_instancename.value,
    c1_id = df_query$c1_id.value,
    c1_name = df_query$c1_name.value,
    c1_mass = df_query$c1_mass.value,
    c2 = df_query$c2.value,
    c2_instancename = df_query$c2_instancename.value,
    c2_id = df_query$c2_id.value,
    c2_name = df_query$c2_name.value, 
    c2_mass = df_query$c2_mass.value
)

print(df_config)


                                                                      c1
1 http://example.com/tutorial2/description/components#orbiter-spacecraft
2 http://example.com/tutorial2/description/components#orbiter-spacecraft
3 http://example.com/tutorial2/description/components#orbiter-spacecraft
4 http://example.com/tutorial2/description/components#orbiter-spacecraft
5 http://example.com/tutorial2/description/components#orbiter-spacecraft
6 http://example.com/tutorial2/description/components#orbiter-spacecraft
7 http://example.com/tutorial2/description/components#orbiter-spacecraft
8 http://example.com/tutorial2/description/components#orbiter-spacecraft
9 http://example.com/tutorial2/description/components#orbiter-spacecraft
     c1_instancename c1_id            c1_name c1_mass
1 orbiter-spacecraft  C.02 Orbiter Spacecraft     0.0
2 orbiter-spacecraft  C.02 Orbiter Spacecraft     0.0
3 orbiter-spacecraft  C.02 Orbiter Spacecraft     0.0
4 orbiter-spacecraft  C.02 Orbiter Spacecraft     0.0


### Tidy Data and Visualize Decompositions

In [13]:
%%R

df_config <- df_config %>%
  mutate(c2_label = paste0(c2_id,": ",c2_instancename)) %>%
  mutate(c1_label = paste0(c1_id,": ",c1_instancename)) 


# just for vis add NA to root
df_vis <- df_config %>%
  add_row(c1=NA,
          c1_instancename=NA,
          c1_id=NA,
          c1_name=NA,
          c1_mass=NA,
          c2=df_config$c1[1],
          c2_instancename=df_config$c1_instancename[1],
          c2_id=df_config$c1_id[1],
          c2_name=df_config$c1_name[1],
          c2_mass=df_config$c1_mass[1],
          c1_type=df_config$c1_type[1],
          c2_label=NA,
          c1_label=df_config$c1_label[1],  
          .before = 1)

plotCollapsibleTreeFromDataframe(df_vis, palette="BluYl", 
                                 parent="c2_label",
                                 child="c1_label",
                                 type="c2_id")


### Prepare a Graph Data for mass rollup calculation

In [14]:
%%R

library(igraph)

df_g <- df_config %>%
  mutate(parent = c1_instancename) %>%
  mutate(child = c2_instancename) %>%
  mutate(type = c2_instancename) %>%
  # mutate(parent = paste0(p_id, ": ", p_instancename)) %>%
  # mutate(child = paste0(c_id, ": ", c_instancename)) %>%  
  select("parent","child","type")

# df_g <- df_g[-1,]

g <- graph_from_data_frame(df_g, 
                           directed = TRUE, 
                           vertices = NULL)



### RollUp Pattern

In [15]:
%%R

root <- V(g)[1]

# Depth-first search is an algorithm to traverse a graph. It starts from a root vertex and tries to go quickly as far from as possible.
order <- dfs(g, V(g)[root], order.out = TRUE)$order

df_mel_before <- igraph::as_data_frame(g, what = "vertices") %>%
  arrange(factor(name, levels = names(order)))%>%
  filter(name !="NA")

# ここに、df_mass として massの値を入れたコンフィグを作る必要がある。

df_mass <-  df_config

df_mass <-  df_mass %>%
  mutate(mass = as.numeric(c2_mass)) %>%
  select(c2,
         c2_instancename,
         c2_id,
         c2_name,
         c1_id,
         mass) %>%  
  add_row(c2=df_mass$c1[1],
          c2_instancename=df_mass$c1_instancename[1],
          c2_id=df_mass$c1_id[1],
          c2_name=df_mass$c1_name[1],
          c1_id=NA,
          mass=as.numeric(df_mass$c1_mass[1]),
          .before = 1)%>%
  arrange(c2_id)


colnames(df_mass) <- c("c_iri","c_instancename","c_id","c_name", "p_id", "mass")

df_mel_before <- left_join(df_mel_before, df_mass, by = c("name"="c_instancename"))


In [16]:
%%R

namekey="name"
masskey="mass"


df_mass_update <- massRollUp(g, root, df_mel_before, namekey = "name",masskey = "mass") %>%
  select(name, mass)


# リーフ数を取得
df_deg <- data.frame(
  name = names(degree(g)),
  degree = degree(g)-1, # num of components is degree(g)-1
  distance = distances(g)[V(g)[1],]
)

# distanceの値で、componentか、workpackageかを切り分けることができる

df_mel_after <- df_mel_before %>% select(-mass)
df_mel_after <- left_join(df_mel_after, df_mass_update, by=c("name"="name")) 
df_mel_after <- left_join(df_mel_after, df_deg, by=c("name"="name")) 

df_mel_after$componenttype <- ifelse(df_mel_after$distance == 0, "system", 
                            ifelse(df_mel_after$distance == 1, "subsystem", 
                                   ifelse(df_mel_after$distance == 2, "assembly", NA)))





In [17]:
%%R

df_table <- df_mel_after %>%
  mutate(totalmass=mass) %>%
  select(c_id, name, totalmass, componenttype) 

df_table$componenttype <- factor(df_table$componenttype)

datatable(df_table, options = list(autoWidth = FALSE, pageLength = -1), filter = list(
  position = 'top', clear = FALSE
))

## Binding Variables : toPython


In [18]:
import rpy2.robjects as robjects
# list(robjects.globalenv.keys())
df_mel_after = robjects.globalenv['df_mel_after']
#type(df_mel_after)
#print(df_mel_after[5][0])
df_parameters['m_dry'] = df_mel_after[5][0]

## deltaV ( Using Python)

### data bindings

In [19]:
orbit_init=float(df_parameters['initOrbit'][0])
orbit_target=float(df_parameters['targetOrbit'][0])


### analysis

In [20]:
import os
import sys

print('getcwd:      ', os.getcwd())
sys.path.append(os.getcwd())
sys.path.append(os.getcwd()+'/src/analysis')

# from src.analysis import analysisOrbit
import analysisOrbit
from astropy import units as u

analysisRocket = analysisOrbit.hohmanTransfer(orbit_init=orbit_init, orbit_target=orbit_target)
# analysisRocket = analysisOrbit.hohmanTransfer(orbit_init=r.initOrbit, orbit_target=r.targetOrbit, m_init=r.m_dry, I_sp=r.I_sp)
# analysisRocket = analysisOrbit.hohmanTransfer(orbit_init=400, orbit_target=35786, m_init=5000, I_sp=300)

total_delta_v = analysisRocket.calculate_delta_v()


print(f"Total delta-v: {total_delta_v}")


getcwd:       /Users/mlab/Workspaces/github/kepler16-using-imce-feat20/kepler16b-using-imce-vocabulary/src/analysis


/Users/mlab/opt/anaconda3/envs/py393/lib/python3.9/site-packages/poliastro/core/maneuver.py:49: NumbaPerformanceWarning: '@' is faster on contiguous arrays, called on (Array(float64, 1, 'A', False, aligned=True), Array(float64, 1, 'A', False, aligned=True))
  r_i = norm(r_i)


Total delta-v: 3853.959363102248 m / s


## Binding Variables: to Python

In [21]:

df_parameters['dv'] = total_delta_v /1000.0 # km/s


## fuelMass (Using R)

### Binding Variables: To R

In [22]:
%%R -i df_parameters

dv <- as.numeric(df_parameters$dv)
m_dry <- as.numeric(df_parameters$m_dry)
I_sp <- as.numeric(df_parameters$I_sp)

/Users/mlab/opt/anaconda3/envs/py393/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "m_fuel". Fall back to string conversion. The error is: <class 'float'>
  warnings.warn('Error while trying to convert '
/Users/mlab/opt/anaconda3/envs/py393/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "m_wet". Fall back to string conversion. The error is: <class 'float'>
  warnings.warn('Error while trying to convert '


In [23]:
%%R

source(searchDirectory(4, "calcWetMass.R", (getwd())))

#print(df_parameters)


m_wet <- calcWetMass(dv, m_dry, I_sp)
m_fuel <- m_wet - m_dry
c(m_dry, m_fuel, m_wet)


[1] 1957.00 4055.68 6012.68


### Binding Variables: to Python

In [24]:
import rpy2.robjects as robjects
# list(robjects.globalenv.keys())

df_parameters['m_wet'] = robjects.globalenv['m_wet']
df_parameters['m_fuel'] = robjects.globalenv['m_fuel']

## Finally, Update OML

### parameter table updated

In [25]:
print(df_parameters)

parameter I_sp        dv initOrbit   m_dry       m_fuel        m_wet  \
0          350  3.853959       400  1957.0  4055.680381  6012.680381   

parameter targetOrbit  
0               35786  


### parameters before and after

In [26]:
df_parameters

parameter,I_sp,dv,initOrbit,m_dry,m_fuel,m_wet,targetOrbit
0,350,3.853959,400,1957.0,4055.680381,6012.680381,35786


In [27]:
df_parameters_after = df_parameters.melt(var_name="parameter", value_name="value_after")

In [28]:
df_parameters_after

,parameter,value_after
0,I_sp,350
1,dv,3.853959
2,initOrbit,400
3,m_dry,1957.0
4,m_fuel,4055.680381
5,m_wet,6012.680381
6,targetOrbit,35786


In [29]:
df_parameters_before

,iri,statevariable,value,parameter,index
0,http://example.com/tutorial2/description/state...,orbiter-spacecraft.delta-v,NaN,dv,0
1,http://example.com/tutorial2/description/state...,orbiter-spacecraft.initial-orbit,400,initOrbit,0
2,http://example.com/tutorial2/description/state...,orbiter-spacecraft.isp,350,I_sp,0
3,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.dry,NaN,m_dry,0
4,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.fuel,NaN,m_fuel,0
5,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.wet,NaN,m_wet,0
6,http://example.com/tutorial2/description/state...,orbiter-spacecraft.target-orbit,35786,targetOrbit,0


In [30]:
import numpy as np

df_parameters_before_after = pd.merge(df_parameters_before, df_parameters_after, left_on='parameter', right_on='parameter', how='left')
df_parameters_before_after = df_parameters_before_after.rename(columns={'value': 'value_before'})

df_parameters_before_after['status'] = np.where((df_parameters_before_after['value_before'])==(df_parameters_before_after['value_after']), 'unchanged', 'changed')



In [31]:
df_parameters_before_after

,iri,statevariable,value_before,parameter,index,value_after,status
0,http://example.com/tutorial2/description/state...,orbiter-spacecraft.delta-v,NaN,dv,0,3.853959,changed
1,http://example.com/tutorial2/description/state...,orbiter-spacecraft.initial-orbit,400,initOrbit,0,400,unchanged
2,http://example.com/tutorial2/description/state...,orbiter-spacecraft.isp,350,I_sp,0,350,unchanged
3,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.dry,NaN,m_dry,0,1957.0,changed
4,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.fuel,NaN,m_fuel,0,4055.680381,changed
5,http://example.com/tutorial2/description/state...,orbiter-spacecraft.mass.wet,NaN,m_wet,0,6012.680381,changed
6,http://example.com/tutorial2/description/state...,orbiter-spacecraft.target-orbit,35786,targetOrbit,0,35786,unchanged


### sending update query

In [32]:
%%R -i df_parameters_before_after

library(tansakusuR)
source(searchDirectory(4, "updateSparqlQuery.R", (getwd())))

df_update <- df_parameters_before_after %>% 
  filter(status == "changed")

endpoint_url <- "http://localhost:3030/tutorial2-tdb/"

for( i in 1:nrow(df_update) ){
  df <- df_update[i,]
  update_value <- as.character(df$value_after)
  update_iri <- df$iri

  df_ret <- updateSparqlQuery(endpoint_url, update_value, update_iri)
  
  ret <- send_update(endpoint_url = endpoint_url, df_ret$query_string_delete_tdb)
  ret <- send_update(endpoint_url = endpoint_url, df_ret$query_string_insert_tdb)
  
}



/Users/mlab/opt/anaconda3/envs/py393/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "value_before". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '
/Users/mlab/opt/anaconda3/envs/py393/lib/python3.9/site-packages/rpy2/robjects/pandas2ri.py:65: UserWarning: Error while trying to convert the column "value_after". Fall back to string conversion. The error is: Series can only be of one type, or None (and here we have <class 'float'> and <class 'str'>). If happening with a pandas DataFrame the method infer_objects() will normalize data types before conversion.
  warnings.warn('Error while trying to convert '



 次のパッケージを付け加えます: ‘tansakusuR’ 

 以下のオブジェクトは ‘package:dplyr’ からマスクされています:

    show_query



## Run Save in OML-Vision

Using OML-Vision GUI, run the save task.
Then you can see the oml file is updated.